In [6]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D

model=Sequential()

model.add(Conv2D(256,(3,3),input_shape=[50,50,1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(64,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(8,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [7]:
model.load_weights('modelAgeGender.h5')

In [8]:
import cv2
import dlib
import numpy as np

In [9]:
cam = cv2.VideoCapture(0)#rtsp://192.168.1.6/live/ch00_1 'https://192.168.1.5:8080/video'
category_dict={0:'13-26_Female', 1:'13-26_Male', 2:'27-40_Female', 3:'27-40_Male', 4:'4-12_Female', 5:'4-12_Male', 6:'40+_Female', 7:'40+_Male'}
face_detector = dlib.get_frontal_face_detector()
margin = 20

In [10]:
while(True):
    ret, feed = cam.read()
    feed = cv2.resize(feed, (852, 480))
    
    faces = face_detector(feed)
    
    try:
        for face in faces:

            x1 = (face.left()) - margin
            y1 = (face.top()) - margin
            x2 = (face.right()) + margin
            y2 = (face.bottom()) + margin

            test_img = feed[y1:y2, x1:x2]

            test_img = cv2.resize(test_img, (50,50))
            test_img = cv2.cvtColor(test_img,cv2.COLOR_BGR2GRAY)
            test_img = test_img/255
            test_img = test_img.reshape(1, 50, 50, 1)

            results = model.predict(test_img)
            label = np.argmax(results, axis=1)[0]
            acc = int(np.max(results, axis=1)[0]*100)

            print(results,label,category_dict[label],acc)

            cv2.rectangle(feed, (x1, y1), (x2, y2),(0, 255, 0), 2)
            cv2.rectangle(feed, (x1-1, y1-20), (x1+99, y1), (0, 255, 0), -1)
            #cv2.putText(feed, "Face", (x1, y1-1), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0 ,0, 255), 2)

            cv2.putText(feed, category_dict[label], (x1, y1-1), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)
            cv2.putText(feed, str(acc) + '%', (x1 + 400, y1-1), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)
    except:
        pass
    
    cv2.imshow('LIVE_FEED',feed)
    k=cv2.waitKey(1)
    if(k==27):
        break
cv2.destroyAllWindows()

[[1.4855447e-11 4.7965700e-14 4.0432092e-18 1.6757953e-11 9.9999130e-01
  8.7425597e-06 6.3366581e-28 1.5705180e-11]] 4 4-12_Female 99
[[3.1465332e-12 2.6583428e-16 1.8542476e-20 9.4341090e-15 9.9982494e-01
  1.7502914e-04 2.6930667e-31 1.4895508e-14]] 4 4-12_Female 99
[[9.0407820e-12 1.8818820e-16 2.1144817e-19 2.6775955e-14 9.9806601e-01
  1.9340108e-03 2.0057533e-31 9.1177134e-14]] 4 4-12_Female 99
[[1.9198327e-11 1.2395080e-16 1.0759566e-20 2.1513574e-15 9.9535793e-01
  4.6420135e-03 2.7421109e-32 6.9364333e-15]] 4 4-12_Female 99
[[6.9383784e-13 8.2856075e-17 1.2855058e-22 5.0638088e-17 9.9947375e-01
  5.2632327e-04 6.4211801e-34 4.8257307e-17]] 4 4-12_Female 99
[[7.4601219e-13 7.2363745e-18 3.2399180e-23 1.7657027e-17 9.9996531e-01
  3.4696568e-05 4.0888575e-34 1.7598524e-17]] 4 4-12_Female 99
[[7.9441939e-12 2.7488848e-18 4.1881217e-21 1.3243257e-15 9.9999297e-01
  7.0179449e-06 4.0676652e-30 1.9332155e-14]] 4 4-12_Female 99
[[6.3256578e-09 2.0954199e-16 6.9011327e-18 3.2013684e-

[[5.8935452e-02 1.2895190e-03 3.4040444e-02 5.7630891e-01 8.5864769e-04
  1.4407195e-11 3.3827465e-09 3.2856700e-01]] 3 27-40_Male 57
[[3.22524365e-03 7.22349650e-05 2.27160999e-04 1.24285996e-01
  3.38672311e-03 8.74009979e-12 2.62600679e-08 8.68802607e-01]] 7 40+_Male 86
[[1.6830080e-04 7.7291112e-04 6.1537904e-01 1.8957710e-02 6.6508545e-04
  2.4488135e-04 2.5500359e-13 3.6381206e-01]] 2 27-40_Female 61
[[4.5758346e-04 1.1375889e-04 1.1131332e-01 2.7724518e-02 1.7743155e-04
  2.9185080e-04 6.5517812e-15 8.5992157e-01]] 7 40+_Male 85
[[1.5920644e-09 2.2837265e-05 1.8484610e-10 4.5126849e-01 4.7876540e-04
  5.4822952e-01 1.7740554e-12 3.4725599e-07]] 5 4-12_Male 54
[[2.3680231e-09 2.3265941e-06 1.9809441e-09 9.9997091e-01 2.3970211e-08
  2.5087087e-05 6.2766411e-12 1.6198592e-06]] 3 27-40_Male 99
[[8.3947338e-09 5.0959407e-08 1.1574031e-08 9.9999988e-01 1.5339747e-11
  1.0118921e-07 1.8312759e-11 1.2124785e-08]] 3 27-40_Male 99
[[1.0583300e-07 2.7916541e-11 4.5127986e-12 9.9999988e-01

[[2.9437086e-05 3.0950709e-14 1.7912307e-11 1.0641673e-06 9.9866951e-01
  1.2997595e-03 2.3841946e-07 3.2754519e-09]] 4 4-12_Female 99
[[9.2203457e-08 1.2801702e-06 1.6083828e-08 9.9910599e-01 9.4877672e-05
  7.9739996e-04 5.3905808e-10 2.2477191e-07]] 3 27-40_Male 99
[[1.7807324e-05 8.1052778e-05 1.0982938e-07 9.7132361e-01 2.1886928e-02
  6.6899122e-03 1.3680614e-09 5.9390703e-07]] 3 27-40_Male 97
[[2.7953301e-06 4.0218892e-04 9.4770991e-10 4.2464412e-03 6.1804950e-01
  3.7729913e-01 1.4848334e-12 2.8262273e-09]] 4 4-12_Female 61
[[3.6940221e-06 3.2701489e-04 1.0832160e-09 2.3271074e-03 9.9284279e-01
  4.4994187e-03 2.2860771e-12 2.5364342e-08]] 4 4-12_Female 99
[[6.9419565e-07 1.8005989e-05 2.1195881e-10 3.3766206e-04 9.9865472e-01
  9.8898122e-04 2.0159516e-13 6.9261121e-09]] 4 4-12_Female 99
[[2.6736549e-07 1.8491754e-05 7.9572050e-11 9.5796051e-05 9.9697006e-01
  2.9153409e-03 7.7891598e-14 3.6458078e-10]] 4 4-12_Female 99
[[5.1502650e-07 2.2411364e-04 2.9886585e-10 4.2928680e-04

[[9.2979348e-01 4.7216590e-06 1.8700448e-04 6.9165185e-02 9.5650776e-06
  8.3994627e-04 6.7085393e-10 6.8685388e-08]] 0 13-26_Female 92
[[2.6394099e-01 1.3577265e-05 4.9818977e-04 7.3126644e-01 1.3511647e-05
  4.2656870e-03 8.6165368e-09 1.6127392e-06]] 3 27-40_Male 73
[[6.1601889e-01 7.6352058e-05 4.9098319e-04 3.7464970e-01 5.3037034e-05
  8.7077590e-03 6.2814500e-08 3.1992804e-06]] 0 13-26_Female 61
[[7.95856237e-01 1.45606638e-04 4.65476682e-04 1.82396188e-01
  1.03936196e-04 2.10297368e-02 8.34814315e-08 2.75511070e-06]] 0 13-26_Female 79
[[9.0689397e-01 1.3272974e-04 2.6457399e-04 8.4807567e-02 1.9738913e-04
  7.6999539e-03 2.2980132e-07 3.5359910e-06]] 0 13-26_Female 90
[[4.8761898e-01 5.4107869e-07 1.2116041e-03 4.2346254e-02 7.2641007e-05
  4.6872646e-01 1.2694565e-08 2.3521361e-05]] 0 13-26_Female 48
[[4.7360167e-01 8.9057698e-07 6.9252066e-03 1.6498198e-01 5.6013771e-05
  3.5439128e-01 2.9428657e-08 4.2929190e-05]] 0 13-26_Female 47
[[3.0086064e-01 1.3327025e-06 6.5896288e-0

[[2.3839544e-07 5.8293224e-09 5.6172571e-08 9.9998116e-01 1.0811603e-09
  3.9540018e-06 2.6143363e-15 1.4698034e-05]] 3 27-40_Male 99
[[1.6715479e-07 1.9661876e-08 5.8078204e-08 9.9996829e-01 3.8889567e-09
  2.7283941e-06 4.8696305e-15 2.8772782e-05]] 3 27-40_Male 99
[[3.2835089e-07 3.3366337e-08 5.6052418e-08 9.9996853e-01 2.4266575e-09
  3.2796299e-06 5.0442210e-15 2.7813732e-05]] 3 27-40_Male 99
[[4.0415730e-06 1.0857158e-07 1.1689783e-07 9.9996758e-01 2.9518581e-09
  1.1419044e-05 2.3651508e-15 1.6678352e-05]] 3 27-40_Male 99
[[4.3705941e-06 4.8040558e-08 5.7918580e-08 9.9997413e-01 2.0903579e-09
  5.7888378e-06 1.0351346e-15 1.5627375e-05]] 3 27-40_Male 99
[[5.3485719e-06 7.5919054e-08 1.7277441e-07 9.9995470e-01 2.3504221e-09
  4.1362423e-06 1.9305022e-15 3.5609559e-05]] 3 27-40_Male 99
[[9.9710757e-01 3.6420883e-04 1.1637478e-05 1.6952337e-03 4.6238512e-07
  8.2081091e-04 2.1723322e-15 2.5685134e-08]] 0 13-26_Female 99
[[9.9975508e-01 3.1519317e-05 5.3438771e-07 1.5537879e-04 3.

...

[[4.84768063e-01 2.09391993e-09 1.41213173e-02 2.58297801e-01
  1.58759015e-08 2.28942633e-01 1.09437690e-06 1.38690425e-02]] 0 13-26_Female 48
[[4.8195893e-01 1.8253533e-09 1.1793286e-02 2.7629587e-01 6.7230363e-09
  2.2079644e-01 1.1040937e-06 9.1543496e-03]] 0 13-26_Female 48
[[5.7318962e-01 1.7390072e-09 1.4587515e-02 1.6747235e-01 9.6867936e-09
  2.3548935e-01 8.5153118e-07 9.2602866e-03]] 0 13-26_Female 57
[[5.9902513e-01 2.3034290e-09 1.5773581e-02 1.3182540e-01 1.6740161e-08
  2.4526387e-01 7.0019769e-07 8.1112273e-03]] 0 13-26_Female 59
[[7.5174916e-01 2.1289683e-09 1.2180149e-02 8.5975230e-02 1.3588264e-08
  1.4570796e-01 3.7712351e-07 4.3870532e-03]] 0 13-26_Female 75
[[8.06760192e-01 1.40806733e-09 1.56443622e-02 6.82967603e-02
  1.51044937e-08 1.07805945e-01 5.28037674e-07 1.49220682e-03]] 0 13-26_Female 80
[[8.3553779e-01 2.1457029e-09 1.9695003e-02 6.1110135e-02 1.9788580e-08
  8.2940727e-02 2.5936924e-07 7.1608071e-04]] 0 13-26_Female 83
[[8.0821437e-01 3.5494221e-09 2.

[[9.4748372e-01 1.0499142e-14 4.8366180e-03 4.7341105e-02 1.2625026e-06
  3.0409517e-09 3.3723519e-04 4.6348390e-12]] 0 13-26_Female 94
[[1.6599084e-04 2.1132962e-10 9.7044778e-01 2.3996211e-06 2.9382445e-02
  2.4205132e-10 1.3641644e-06 1.5428381e-11]] 2 27-40_Female 97
[[2.4934651e-04 4.0302486e-10 9.7498113e-01 2.5700313e-06 2.4766153e-02
  2.9625283e-10 8.7003787e-07 1.3448083e-11]] 2 27-40_Female 97
[[2.0503972e-04 1.7912001e-09 9.2574441e-01 7.7669747e-06 7.4042007e-02
  6.5064049e-10 7.7085343e-07 3.2557939e-11]] 2 27-40_Female 92
[[2.6109387e-04 1.2200206e-09 8.6205453e-01 6.2792365e-06 1.3767758e-01
  8.6867585e-10 4.7763376e-07 2.2299987e-11]] 2 27-40_Female 86
[[2.8898829e-04 1.2811410e-09 6.2169313e-01 1.4990263e-05 3.7800252e-01
  1.8128463e-09 4.1324398e-07 2.5709556e-11]] 2 27-40_Female 62
[[2.6806595e-04 8.8802687e-10 6.7961061e-01 1.3395579e-05 3.2010752e-01
  1.3342054e-09 4.2456321e-07 3.5780691e-11]] 2 27-40_Female 67
[[1.8079524e-04 5.9996230e-10 7.4842232e-01 2.55

[[5.5944300e-01 7.0575143e-11 2.0681462e-08 4.4055688e-01 1.6636759e-16
  8.5412843e-08 1.5992285e-15 5.1743969e-12]] 0 13-26_Female 55
[[2.3285393e-01 1.1533491e-09 7.6563161e-08 7.6714581e-01 6.0244292e-16
  1.7645661e-07 2.4258407e-15 1.1744329e-10]] 3 27-40_Male 76
[[6.52540565e-01 1.19522765e-08 7.57789280e-07 3.47458541e-01
  2.74915406e-15 1.77164935e-07 1.24033853e-13 9.50007073e-10]] 0 13-26_Female 65
[[8.9736074e-01 2.1057345e-09 2.0480034e-06 1.0263649e-01 1.8997346e-15
  7.8810484e-07 1.0695897e-13 4.6527707e-10]] 0 13-26_Female 89
[[9.6082681e-01 1.4331020e-09 2.7576191e-06 3.9169386e-02 1.8269143e-15
  1.0766342e-06 3.8835319e-14 1.1006517e-09]] 0 13-26_Female 96
[[2.2119334e-01 1.1133153e-09 1.5572016e-06 7.7880180e-01 1.1260920e-14
  3.1935249e-06 7.7211761e-13 1.2929077e-07]] 3 27-40_Male 77
[[1.9431229e-06 1.6344410e-08 2.0123756e-05 9.9923408e-01 8.1485539e-14
  6.9954228e-07 8.4350325e-11 7.4322836e-04]] 3 27-40_Male 99
[[1.61699219e-07 4.63423344e-09 1.09359535e-05

[[9.9995589e-01 2.6361397e-07 3.1310168e-10 4.3902848e-05 3.6506336e-11
  5.9184748e-13 5.5390140e-15 1.0401057e-10]] 0 13-26_Female 99
[[9.9997509e-01 3.7211835e-06 1.2181146e-10 2.1262167e-05 1.9049931e-10
  5.4003991e-13 3.5510531e-15 1.5747648e-10]] 0 13-26_Female 99
[[9.2886186e-01 4.1626356e-04 7.8115525e-10 7.0721827e-02 4.2296866e-10
  4.3981173e-11 5.4983746e-15 9.2856453e-08]] 0 13-26_Female 92
[[1.5163761e-01 5.0913764e-06 1.5594674e-10 8.4835732e-01 2.4865414e-11
  7.1137479e-10 9.3235279e-17 1.5637282e-08]] 3 27-40_Male 84
[[1.17646385e-04 1.76885628e-09 3.54643259e-11 9.99882221e-01
  2.19455625e-13 5.98502865e-08 2.09410295e-14 1.10631895e-08]] 3 27-40_Male 99
[[8.0827440e-06 3.4118815e-11 2.6451196e-12 9.9999177e-01 6.5732185e-13
  8.2010366e-08 9.8209201e-15 9.1571650e-10]] 3 27-40_Male 99
[[3.8241706e-06 2.3055514e-10 7.8447934e-12 9.9999595e-01 6.5963211e-12
  2.8035691e-07 1.5347791e-15 1.5842470e-08]] 3 27-40_Male 99
[[1.6351386e-06 1.3821970e-09 6.1441108e-12 9.99

[[3.9353016e-01 1.3950277e-06 5.6077319e-04 6.0537440e-01 1.2896742e-08
  5.2917570e-11 7.3769049e-09 5.3321529e-04]] 3 27-40_Male 60
[[9.9999738e-01 2.0122711e-11 2.3971647e-06 2.0420505e-07 7.7169326e-10
  3.0033709e-10 1.2423619e-10 9.5608753e-13]] 0 13-26_Female 99
[[2.1591573e-03 6.4762601e-12 1.6477927e-06 2.3619188e-08 9.9783605e-01
  4.7620682e-07 2.6647758e-06 2.4886404e-14]] 4 4-12_Female 99
[[9.0050712e-08 2.2579392e-16 1.0660235e-09 6.7050189e-12 9.9999988e-01
  7.1645367e-10 1.3364575e-10 3.0064438e-14]] 4 4-12_Female 99
[[8.6805049e-11 4.4201044e-13 1.7828678e-06 2.8881972e-08 2.4791915e-11
  1.7386873e-13 5.1120097e-10 9.9999821e-01]] 7 40+_Male 99
[[9.9986899e-01 9.0407352e-12 1.7229562e-05 6.2233390e-08 1.0657352e-04
  6.7528399e-06 4.4563151e-08 3.6532984e-07]] 0 13-26_Female 99
[[9.9977750e-01 1.1335724e-12 1.7907456e-05 5.8338691e-07 2.0249558e-04
  9.9686383e-07 2.5943541e-07 2.9798619e-07]] 0 13-26_Female 99
[[1.7106763e-01 5.4704975e-07 6.0132289e-01 1.6727419e-0